#### **PC3**

## LECTURA DE DATOS


Se añade la data

In [121]:
import pandas as pd
reserves= pd.read_csv('https://github.com/Tracy-12/PRACTICA-3/raw/refs/heads/main/DATA1_pc3.csv')
energy= pd.read_csv('https://github.com/Tracy-12/PRACTICA-3/raw/refs/heads/main/DATA2_pc3.csv')
electricity= pd.read_csv('https://github.com/Tracy-12/PRACTICA-3/raw/refs/heads/main/DATA3_pc3.csv')
education= pd.read_csv('https://github.com/Tracy-12/PRACTICA-3/raw/refs/heads/main/DATA4_pc3.csv')

## PASO 1 : Limpieza y formateo de los data frames

Primero, se muestra el nombre de las columnas para saber cuales se eliminan y cuales se cambian de nombre en cada data frame

In [122]:
reserves.columns,energy.columns,electricity.columns,education.columns

(Index(['name', 'slug', 'value', 'date_of_information', 'ranking', 'region'], dtype='object'),
 Index(['name', 'slug', ' Btu/person', 'date_of_information', 'ranking',
        'region'],
       dtype='object'),
 Index(['name', 'slug', ' kW', 'date_of_information', 'ranking', 'region'], dtype='object'),
 Index(['name', 'slug', '% of GDP', 'date_of_information', 'ranking', 'region'], dtype='object'))

Luego se eliminan las columnas no deseadas y se cambia el nombre de las columnas

In [123]:
reserves,energy,electricity,education = [df.drop(columns=['ranking','date_of_information','slug']) for df in [reserves,energy,electricity,education]]
dataframes = [reserves, energy, electricity, education]
cambios_nombres = {
    'reserves': {'name': 'Country', 'value': 'Reserves', 'region':'Region'},
    'energy': {'name': 'Country', ' Btu/person': 'Energy','region':'Region'},
    'electricity': {'name': 'Country',' kW': 'Electricity','region':'Region'},
    'education': {'name': 'Country','% of GDP': 'Education','region':'Region'}
}
for df in dataframes:
    df_name = [name for name, var in globals().items() if var is df][0]
    if df_name in cambios_nombres:
        df.rename(columns=cambios_nombres[df_name], inplace=True)


Por ultimo en esta seccion se limpia los espacios en cada columna y se convierte los datos a valores numericos

In [124]:
dataframes_para_limpiar= {
    'reserves': 'Reserves','energy': 'Energy','electricity': 'Electricity','education': 'Education'
}
for df_name, column_name in dataframes_para_limpiar.items():
    df = globals()[df_name]
    df[column_name] = df[column_name].astype(str).str.replace(pat=r'\D', repl='', regex=True)
    df[column_name] = pd.to_numeric(df[column_name])
#Para comprobar que se convirtio los datos a numeros se ejecuta:
reserves.info(),electricity.info(),energy.info(),education.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Country   195 non-null    object
 1   Reserves  195 non-null    int64 
 2   Region    195 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Country      211 non-null    object
 1   Electricity  211 non-null    int64 
 2   Region       211 non-null    object
dtypes: int64(1), object(2)
memory usage: 5.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  195 non-null    object
 1   Energy   195 non-null    int64 
 2   Region   195 non-null    object
dtypes: 

(None, None, None, None)

## PASO 2: Union de data frames

Primero se verifica el numero de columnas y filas que contiene cada data frame

In [125]:
reserves.shape,energy.shape,electricity.shape,education.shape

((195, 3), (195, 3), (211, 3), (197, 3))

Se busca que **paises** tienen **en comun** entre las 4 variables

In [126]:
# Se obtiene el conjunto de países únicos de cada data frame
paises_reserves = set(reserves['Country'])
paises_energy = set(energy['Country'])
paises_electricity = set(electricity['Country'])
paises_education = set(education['Country'])
# Luego se encuentra la intersección de los cuatro data frames
paises_comunes = list(paises_reserves.intersection(paises_energy, paises_electricity, paises_education))
# Y se muestra la interseccion
print(f"Número de países en la interseccion: {len(paises_comunes)}")

Número de países en la interseccion: 176


Luego se busca los **países únicos** por data frame

In [127]:
paises_comunes=set(paises_comunes)
paises_unicos_por_df = {
    'Reserves': paises_reserves - paises_comunes,
    'Energy': paises_energy - paises_comunes,
    'Electricity': paises_electricity - paises_comunes,
    'Education': paises_education - paises_comunes
}
for df_name, unique_countries in paises_unicos_por_df.items():
    print(f"Países que están solo en {df_name}:", list(unique_countries)[:5])

Países que están solo en Reserves: ['North Macedonia', 'Palau', 'Anguilla', 'Yemen', 'Micronesia, Federated States of']
Países que están solo en Energy: ['North Macedonia', 'Yemen', 'American Samoa', 'Guam', 'Montenegro']
Países que están solo en Electricity: ['North Macedonia', 'New Caledonia', 'Nauru', 'Gibraltar', 'Yemen']
Países que están solo en Education: ['Nauru', 'Palau', 'Anguilla', 'Micronesia, Federated States of', 'Montserrat']


*Como no hay algun pais en común entre los cuatro data frames que tenga una escritura similar, no se puede recuperar más datos*

Se observa que como maximo se obtendra **176 paises**, luego se juntan los data frames

In [128]:
dfs = [reserves, energy, electricity, education]
from functools import reduce
merged_data = reduce(lambda left, right: pd.merge(left, right, on=['Country', 'Region'], how='inner'), dfs)
merged_data

Country       Reserves                   Region     Energy  \
0                    China  3450000000000  East and Southeast Asia  113805000   
1                    Japan  1295000000000  East and Southeast Asia  129504000   
2              Switzerland   863892000000                   Europe   99578000   
3            United States   773426000000            North America  278474000   
4                    India   627793000000               South Asia   25179000   
..                     ...            ...                      ...        ...   
171                Burundi       90350000                   Africa     946000   
172                   Togo       77800000                   Africa    4538000   
173           Burkina Faso       49000000                   Africa    3481000   
174  Sao Tome and Principe       46247000                   Africa    8875000   
175                Somalia       30450000                   Africa     649000   

     Electricity  Education  
0     2949000000         36  
1      361617000         34  
2       26502000         52  
3     1235000000         61  
4      499136000         45  
..           ...        ...  
171       131000         50  
172       326000         40  
173       749000         55  
174        29000         50  
175       156000          3  

[176 rows x 6 columns]

## PASO 3: Promedio, minimo y maximo de cada variable por region

Se calcula la **media, minimo y maximo** para cada variable por **region**

In [129]:
stats = merged_data.groupby('Region')[['Reserves', 'Energy', 'Electricity', 'Education']].agg(['mean', 'min', 'max'])
stats

Reserves                               \
                                           mean           min            max   
Region                                                                         
Africa                             7.087900e+09      30450000    81217000000   
Australia and Oceania              1.155909e+10     396530000    61703000000   
Central America and the Caribbean  4.447386e+09     183530000    21311000000   
Central Asia                       8.023244e+10    3237000000   597217000000   
East and Southeast Asia            3.723743e+11     781995000  3450000000000   
Europe                             8.739651e+10    1223000000   863892000000   
Middle East                        1.002204e+11    1323000000   457949000000   
North America                      3.684313e+11  117551000000   773426000000   
South America                      4.994452e+10     895275000   355021000000   
South Asia                         8.624625e+10     590523000   627793000000   

                                         Energy                       \
                                           mean       min        max   
Region                                                                 
Africa                             1.412284e+07    649000   98847000   
Australia and Oceania              6.221671e+07   5655000  223158000   
Central America and the Caribbean  5.587300e+07   3486000  153952000   
Central Asia                       1.039161e+08  16192000  261142000   
East and Southeast Asia            1.186874e+08   6825000  643259000   
Europe                             1.131422e+08  27407000  234698000   
Middle East                        2.406509e+08  14991000  814308000   
North America                      2.158707e+08  57539000  311599000   
South America                      4.641467e+07  25733000   78496000   
South Asia                         2.350638e+07   3380000   64082000   

                                    Electricity                         \
                                           mean        min         max   
Region                                                                   
Africa                             4.619820e+06      29000    65989000   
Australia and Oceania              1.706100e+07      34000   108193000   
Central America and the Caribbean  1.580500e+06      41000     6581000   
Central Asia                       4.239578e+07    3944000   301926000   
East and Southeast Asia            2.108446e+08     277000  2949000000   
Europe                             4.027509e+07     779000   275658000   
Middle East                        3.381729e+07     352000   119620000   
North America                      5.008580e+08  105586000  1235000000   
South America                      3.550408e+07     259000   240251000   
South Asia                         7.211612e+07     432000   499136000   

                                   Education           
                                        mean min  max  
Region                                                 
Africa                             42.680000   3   96  
Australia and Oceania              63.142857  22  128  
Central America and the Caribbean  45.550000  14   87  
Central Asia                       43.333333  28   62  
East and Southeast Asia            36.333333  14   63  
Europe                             51.514286  31   72  
Middle East                        45.285714  17   78  
North America                      52.000000  43   61  
South America                      48.083333  13   98  
South Asia                         38.500000  19   70

Se mejora la presentacion de la tabla

*Primero se separa los subinidices (mean,min,max) que se generan por variable y luego se une cada subindice con el nombre de su variable*

In [130]:
stats.columns=['_'.join(col) for col in stats.columns.to_flat_index()]
#Y luego se convierte la columna region de indice a columna regular
stats.reset_index(drop=False, inplace=True)
stats

Region  Reserves_mean  Reserves_min  \
0                             Africa   7.087900e+09      30450000   
1              Australia and Oceania   1.155909e+10     396530000   
2  Central America and the Caribbean   4.447386e+09     183530000   
3                       Central Asia   8.023244e+10    3237000000   
4            East and Southeast Asia   3.723743e+11     781995000   
5                             Europe   8.739651e+10    1223000000   
6                        Middle East   1.002204e+11    1323000000   
7                      North America   3.684313e+11  117551000000   
8                      South America   4.994452e+10     895275000   
9                         South Asia   8.624625e+10     590523000   

    Reserves_max   Energy_mean  Energy_min  Energy_max  Electricity_mean  \
0    81217000000  1.412284e+07      649000    98847000      4.619820e+06   
1    61703000000  6.221671e+07     5655000   223158000      1.706100e+07   
2    21311000000  5.587300e+07     3486000   153952000      1.580500e+06   
3   597217000000  1.039161e+08    16192000   261142000      4.239578e+07   
4  3450000000000  1.186874e+08     6825000   643259000      2.108446e+08   
5   863892000000  1.131422e+08    27407000   234698000      4.027509e+07   
6   457949000000  2.406509e+08    14991000   814308000      3.381729e+07   
7   773426000000  2.158707e+08    57539000   311599000      5.008580e+08   
8   355021000000  4.641467e+07    25733000    78496000      3.550408e+07   
9   627793000000  2.350638e+07     3380000    64082000      7.211612e+07   

   Electricity_min  Electricity_max  Education_mean  Education_min  \
0            29000         65989000       42.680000              3   
1            34000        108193000       63.142857             22   
2            41000          6581000       45.550000             14   
3          3944000        301926000       43.333333             28   
4           277000       2949000000       36.333333             14   
5           779000        275658000       51.514286             31   
6           352000        119620000       45.285714             17   
7        105586000       1235000000       52.000000             43   
8           259000        240251000       48.083333             13   
9           432000        499136000       38.500000             19   

   Education_max  
0             96  
1            128  
2             87  
3             62  
4             63  
5             72  
6             78  
7             61  
8             98  
9             70

## PASO 4: Creacion de intervalos por variable

In [131]:
!pip install mapclassify

In [132]:
import mapclassify
import numpy as np
columnas = ['Electricity', 'Education', 'Energy', 'Reserves']
# Número de intervalos
k = 5
# Clasificación y creación de nuevas columnas
for col in columnas:
    eq_classifier = mapclassify.FisherJenks(merged_data[col], k=k)
    merged_data[f'{col}_fj5'] = eq_classifier.yb
# Se ordena los nombres de las columnas en la tabla para una mejor presentacion
merged_data = merged_data[['Country', 'Region', 'Electricity', 'Electricity_fj5', 'Education', 'Education_fj5', 'Energy', 'Energy_fj5', 'Reserves', 'Reserves_fj5']]
merged_data

Country                   Region  Electricity  \
0                    China  East and Southeast Asia   2949000000   
1                    Japan  East and Southeast Asia    361617000   
2              Switzerland                   Europe     26502000   
3            United States            North America   1235000000   
4                    India               South Asia    499136000   
..                     ...                      ...          ...   
171                Burundi                   Africa       131000   
172                   Togo                   Africa       326000   
173           Burkina Faso                   Africa       749000   
174  Sao Tome and Principe                   Africa        29000   
175                Somalia                   Africa       156000   

     Electricity_fj5  Education  Education_fj5     Energy  Energy_fj5  \
0                  4         36              1  113805000           1   
1                  2         34              1  129504000           1   
2                  0         52              2   99578000           1   
3                  3         61              3  278474000           2   
4                  2         45              2   25179000           0   
..               ...        ...            ...        ...         ...   
171                0         50              2     946000           0   
172                0         40              1    4538000           0   
173                0         55              2    3481000           0   
174                0         50              2    8875000           0   
175                0          3              0     649000           0   

          Reserves  Reserves_fj5  
0    3450000000000             4  
1    1295000000000             3  
2     863892000000             3  
3     773426000000             3  
4     627793000000             2  
..             ...           ...  
171       90350000             0  
172       77800000             0  
173       49000000             0  
174       46247000             0  
175       30450000             0  

[176 rows x 10 columns]

In [133]:
merged_data.to_csv('merged_data.csv', index=True)
stats.to_csv('stats.csv', index=True)

## Adicional

Para comprobar que la distribucion elegida es la correcta, se puede aplicar el codigo siguiente cambiando el nombre de la variable en **theVar**, donde se demuestra que la opcion de FisherJenks tiene el menor valor de ADCM

In [134]:
import pandas as pd
import mapclassify
import numpy as np
variables = ['Electricity', 'Education', 'Energy', 'Reserves']

# Diccionario para almacenar los resultados de ADCM por variable
adcm_results = {}
# Número de intervalos
K = 5
# Itera sobre cada variable
for var in variables:
    # Selecciona la columna de la variable actual
    theVar = merged_data[var]
    # Aplica los diferentes clasificadores
    ei5 = mapclassify.EqualInterval(theVar, k=K)
    msd = mapclassify.StdMean(theVar)
    q5 = mapclassify.Quantiles(theVar, k=K)
    mb5 = mapclassify.MaximumBreaks(theVar, k=K)
    ht = mapclassify.HeadTailBreaks(theVar)
    fj5 = mapclassify.FisherJenks(theVar, k=K)
    jc5 = mapclassify.JenksCaspall(theVar, k=K)
    mp5 = mapclassify.MaxP(theVar, k=K)
    # Lista de los clasificadores aplicados
    class5 = [q5, ei5, msd, ht, mb5, fj5, jc5, mp5]
    # Recopila los valores de ADCM para cada clasificador en la variable actual
    fits = np.array([c.adcm for c in class5])
    # Crea un DataFrame temporal para los resultados de esta variable
    adcms_var = pd.DataFrame(fits)
    adcms_var['classifier'] = [c.name for c in class5]
    adcms_var.columns = ['ADCM', 'Classifier']
    # Almacena el DataFrame de resultados en el diccionario, usando el nombre de la variable como clave
    adcm_results[var] = adcms_var.set_index('Classifier')['ADCM']
# Crea el DataFrame final combinando los resultados de ADCM de todas las variables
final_adcm_table = pd.DataFrame(adcm_results)
# Tabla final
display(final_adcm_table)

Electricity  Education        Energy      Reserves
Classifier                                                         
Quantiles       6.521500e+09      759.0  4.820336e+09  9.678111e+12
EqualInterval   4.566754e+09      966.0  5.973337e+09  8.622678e+12
StdMean         5.327909e+09     1251.0  7.108938e+09  9.084147e+12
HeadTailBreaks  2.150914e+09      962.0  3.562189e+09  4.046516e+12
MaximumBreaks   3.056504e+09     2043.0  9.071188e+09  7.008030e+12
FisherJenks     1.993811e+09      701.0  3.377008e+09  3.663000e+12
JenksCaspall    4.716748e+09      727.0  3.475282e+09  5.721590e+12
MaxP            6.971762e+09      879.0  5.251100e+09  1.095231e+13